# Script to plot figure 9: model and proxy correlations

In [15]:
#Input Data: 
modelDataPath = 'Data/Model/' #netcdf of model data. Unique my model/season. Each contains multiple climate variables

#Load Packages
import cartopy.crs         as ccrs
import cartopy.util        as cutil
import matplotlib.pyplot   as plt
import matplotlib.gridspec as gridspec
from   matplotlib.colors   import LinearSegmentedColormap
import numpy               as np
import os
import pandas              as pd 
import regionmask          as rm
from   scipy.stats         import pearsonr 
import warnings
import xarray              as xr

warnings.filterwarnings('ignore')
print("Packages Loaded")

Packages Loaded


In [2]:
#Set Working Directory
wd = '/Volumes/GoogleDrive/My Drive/zResearch/Manuscript/2021_HoloceneHydroclimate/2021_HoloceneHydroclimate'
os.chdir(wd)
print('Working directory set to: '+wd)

Working directory set to: /Volumes/GoogleDrive/My Drive/zResearch/Manuscript/2021_HoloceneHydroclimate/2021_HoloceneHydroclimate


In [12]:
#Plot Agreement Settings

#Model Data to Plot
times = [0,12]
seasons   = ['ANN']
v_1   = 'pre'
v_2   = 'tas'
regrid = True

#True/False to save/not save
save = True 

#Plot Settings
font = 'Times New Roman'
plt.rcParams['font.family'   ] = font
plt.rcParams['axes.facecolor'] ='white'
plt.rcParams['axes.linewidth'] = 0.5
plt.rcParams['axes.edgecolor'] = 'k'
plt.rcParams.update({'font.size': 10})

#Print summary
if save: print("Settings chosen to save "+str(seasons)+" "+v_1+" vs "+v_2)
else:    print("Settings chosen to plot "+str(seasons)+" "+v_1+" vs "+v_2)
    

#Load data
modelData = {}
for model in ['hadcm','trace']:
    modelData[model] = {}
    for szn in seasons:
        if regrid: end = '_regrid.nc'
        else:      end =  '.nc'
        modelData[model][szn] = xr.open_dataset(modelDataPath+model+'/'+model+'_'+szn+end,decode_times=False)
    print(model+" "+str(sorted([i for i in modelData[model][szn].data_vars]))+' loaded')

Settings chosen to save ['ANN'] pre vs tas
hadcm ['evp_regrid', 'p-e_regrid', 'pre_regrid', 'tas_regrid'] loaded
trace ['evp_regrid', 'p-e_regrid', 'pre_regrid', 'tas_regrid'] loaded


In [13]:
#Define Function for calculating grid cell correlation values

def calcCorrelation (inData,model1,model2,var1,var2,season,t0=0,t1=12,mask=False,regrid=True):
    if regrid == False:
        modelData1 = inData[model1][season][var1]
        modelData2 = inData[model2][season][var2]
        lats,lons = modelData1.lat,modelData1.lon
    else: 
        modelData1 = inData[model1][season][var1+'_regrid']
        modelData2 = inData[model2][season][var2+'_regrid']
        lats,lons = modelData1.lat_regrid,modelData1.lon_regrid
    t_0 = np.argmin(np.abs(t0*1000 - modelData1.time.data))   
    t_1 = np.argmin(np.abs(t1*1000 - modelData1.time.data))
    out = np.full(np.shape(modelData1)[1:3],np.NaN)
    for lat in range(len(lats)):
        for lon in range(len(lons)):
            out[lat,lon] = pearsonr(modelData1[t_0:t_1,lat,lon].data,modelData2[t_0:t_1,lat,lon].data)[0]
    return(out)

print('Function Created')

Function Created


In [16]:
#Calculate using function

#rVals = calcCorrelation(modelData,m_1,m_2,v_1,v_2,szn,times[0],times[1])
rVals1 = calcCorrelation(modelData,'hadcm','hadcm',v_1,v_2,szn,times[0],times[1])
rVals2 = calcCorrelation(modelData,'trace','trace',v_1,v_2,szn,times[0],times[1])
rVals  = np.mean([rVals2,rVals1],axis=0)

print("Calculating Correlations...  Done")

Calculating Correlations...  Done
